In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from historical.readData.estimateData import read
from historical.readData.estimateData import search_travels
from historical.readData.estimateData import estimate
from historical.readData.estimateData import stops_distance

from historical.readData.travels import calcula_dist_shape
from historical.readData.travels import haversine2
import historical.readData.travels as tv

import networkx as nx


In [2]:
# selected_line = ['8700-10-1']
selected_line = ['8700-10-1',                
                '7545-10-1',
                '7545-10-0',
                '6450-10-1',
                '6450-10-0',
                '3301-10-1',
                '3301-10-0',
                '2290-10-1',
                '2290-10-0',
                '574J-10-1',
                '574J-10-0',
                '477P-10-1',
                '477P-10-0',
                '351F-10-1',
                '351F-10-0']

In [3]:

pth_files_GTFS = "./historical/readData/"

trips = pd.read_csv(pth_files_GTFS + 'trips.txt', sep=',')
shapes = pd.read_csv(pth_files_GTFS + 'shapes.txt', sep=',')
stops = pd.read_csv(pth_files_GTFS + 'stops.txt', sep=',')
stop_times = pd.read_csv(pth_files_GTFS + 'stop_times.txt', sep=',')   

periods = ['morning', 'm_peak', 'i_peak', 'a_peak', 'night']


In [4]:
# Calculate the distance betwen bus stops
def get_distance_stops(distances, stop_buses_trip):

    # Distance between all the bus stops
    lat1 = stop_buses_trip.iloc[0, 7]
    lon1 = stop_buses_trip.iloc[0, 8]
    totalcal = [0.]
    dist = [0.]
    for lat2, lon2 in zip(stop_buses_trip.iloc[1:, 7], stop_buses_trip.iloc[1:, 8]):
        d = haversine2(lat1, lon1, lat2, lon2)
        d = d * 1000
        dist.append(d)
        totalcal.append(totalcal[-1] + d)
        lat1 = lat2
        lon1 = lon2
    distance_all_stops = [stop_buses_trip.iloc[:,3], stop_buses_trip.iloc[:, 7], stop_buses_trip.iloc[:, 8], totalcal]

    return distance_all_stops

# Create the graph of a trip
def get_graph(select_lines, stop_buses_trip, distances):
    """

    :type stop_times: object
    """
    # Create a object Graph with the library networkX
    G = nx.Graph()

    # Add the nodes in the graph with each one of their attributes
    for node in range(len(stop_buses_trip)):
        G.add_node(stop_buses_trip.loc[node, "stop_id"], id=stop_buses_trip.loc[node, "stop_id"], lat=stop_buses_trip.loc[node, "stop_lat"],
            lon=stop_buses_trip.loc[node, "stop_lon"], stop_seq=stop_buses_trip.loc[node, "stop_sequence"])

    distance_all_stops_p, distance_all_stops_mp = stops_distance(select_lines, trips, shapes, stops, stopid)    
    distance_all_stops = get_distance_stops(distances, stop_buses_trip)
    
    

    # Add the edges with other attributes related to them
    for node in range(len(stop_buses_trip)-1):
#         print(distance_all_stops[0][node])
#         print(distance_all_stops_mp[node])
        G.add_edge(stop_buses_trip.loc[node, "stop_id"], stop_buses_trip.loc[node+1, "stop_id"], distance=distance_all_stops_mp[node])
    
#     G = nx.path_graph(38)
#     nx.draw(G)
#     plt.show()


    return G

In [5]:
# if __name__ == '__main__':

distances = calcula_dist_shape(selected_line, trips, shapes)

bus_network_graph = nx.MultiGraph
array_graph_temporal = []

for line in zip(selected_line):
    print(line[0])
    stop_buses_line = stop_times.loc[stop_times['trip_id'].isin(line)]
    stop_buses_line = pd.merge(stop_buses_line, stops, on="stop_id", how="left")
    graph_temporal = get_graph(line[0], stop_buses_line, distances[line[0]])
    
    array_graph_temporal.append(graph_temporal)

bus_network_graph = nx.disjoint_union_all(array_graph_temporal)
print(nx.info(bus_network_graph))

8700-10-1


NameError: name 'stopid' is not defined

In [ ]:
print(bus_network_graph.nodes(data=True))    